# First Assignment

# First question

The objective of this code is to scrape a wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to get the boroughs and neighboroughs in Toronto 

In [200]:
import requests
import pandas as pd
import re

we request the page with the library **requests** 

In [201]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&action=edit"
response = requests.get(url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&action=edit")

In [202]:
response.status_code

200

the request worked!

We need to split the text in order to identify the valued information. 

In [203]:
data_info = response.text.split("{")[16].split("\n")[5:]

In [204]:
data_info[0:8]

['| M1A', '| Not assigned', '| ', '|-', '| M2A', '| Not assigned', '| ', '|-']

In [205]:
Postal_code=list()
Borough=list()
Neighborhood=list()

for i in range(0,len(data_info)-2,4):
    Postal_code = Postal_code + [data_info[i].replace("| ","")]
    Borough = Borough + [data_info[i+1].replace("| ","")]
    Neighborhood = Neighborhood + [data_info[i+2].replace("| ","")]

In [211]:
df=pd.DataFrame({"PostalCode":Postal_code,"Borough":Borough,"Neighborhood":Neighborhood})
df = df[~(df.Borough=='Not assigned')].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


let's put a comma between Neighborhood from the same Postal Code

In [207]:
aggregation = {"Neighborhood": lambda x :  ", ".join(x)}
df = df.groupby(["PostalCode","Borough"]).agg(aggregation).reset_index()

df.Neighborhood = [x.replace(" /",",") for x in df.Neighborhood]

In [208]:
df[df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [209]:
df[df.Neighborhood=='Not assigned'].index.values

array([], dtype=int64)

it seems that there is no Neighborhood with **Not assigned**. However, if there were this would be the code to change their values.

In [210]:
for values in df.index.values:
    PC,Bor,Neigh = df.loc[values]
    if Neigh == 'Not assigned':
        df.loc[values,"Neighborhood"]=Bor

In [191]:
df.shape

(103, 3)